In [31]:
import numpy as np
import itertools
from quimb.tensor.fermion_2d_tebd import Hubbard2D, SimpleUpdate
from pyblock3.algebra import fermion_operators as ops

t=1
u=4
Lx = 4
Ly = 1
#mu = -0.9
mu = 0
Ham = Hubbard2D(t, u, Lx, Ly, mu=mu)
#efci = -5.702748483462062

state_array = np.zeros([Lx,Ly])
state_array[0,0] = state_array[2,0] = 1
state_array[1,0] = state_array[3,0] = 2

from quimb.tensor.fermion_2d import gen_mf_peps

psi = gen_mf_peps(state_array) # this is now a 2d mean field PEPS


sz = ops.measure_sz()
nop = ops.count_n()

sz_ops = {(ix,iy): sz for ix, iy in itertools.product(range(Lx), range(Ly))}
n_ops = {(ix,iy): nop for ix, iy in itertools.product(range(Lx), range(Ly))}


book = {(0,0):"0", (0,1):"+-", (1,0):"+", (1,1):"-"}

def print_block(blk):
    qlab= [iq.n for iq in blk.q_labels]
    ind = np.where(abs(np.asarray(blk)) > 1e-20)
    need_print = False
    for ixs in zip(*ind):
        val = np.asarray(blk)[ixs]
        desc = "|"
        for ix, s in enumerate(ixs):
            desc += book[(qlab[ix], s)]
            if ix != len(ixs)-1:
                desc += ","
            else:
                desc += ">"
            
        
        if (desc.count("+"), desc.count("-")) != (2,2):
            desc = str(val) + desc
            print(desc)
            need_print = True
    return need_print


ket = psi.contract(all)
tsr = ket.data.to_sparse()
for iblk in tsr:
    print(np.asarray(iblk))
   

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [1. 0.]]

  [[0. 0.]
   [0. 0.]]]]


In [33]:
from pyblock3.algebra.fermion import SparseFermionTensor
np.random.seed(3)

            
def callback(su):
    psi1 = su.get_state()
    #for itsr in psi1:
     #   itsr.data.data[abs(itsr.data.data)<1e-9] =0
    state = psi1.contract(all)
    tsr = state.data.to_sparse()
    
    need_print = False
    for blk in tsr.blocks:
        need_print = need_print or print_block(blk)
    if need_print:
        print("Ending Cycle%i"%su._n)

            
su = SimpleUpdate(
    psi,
    Ham,
    chi=128,  # boundary contraction bond dim for computing energy
    D = 4,
    compute_energy_every=100,
    compute_energy_per_site=False,
    keep_best=True,
    ordering = 'sort',
    gauge_smudge = 1e-6,
    callback=callback#,
    #gate_opts = {'cutoff': 1e-6}
)
tau = 0.01
su.evolve(278, tau=tau)

second_dense = min(Lx, Ly)>1
sz_expecs = su.get_state().compute_local_expectation(sz_ops, return_all=True, second_dense=second_dense,normalized=True)
n_expecs = su.get_state().compute_local_expectation(n_ops, return_all=True, second_dense=second_dense,normalized=True)

print(su.get_state()[0,0].shape)
for ix, iy in itertools.product(range(Lx), range(Ly)):
    print("(%i, %i): SZ=%.2f, N=%.2f"%(ix,iy,sz_expecs[(ix,iy)][0]/sz_expecs[(ix,iy)][1], n_expecs[(ix,iy)][0]/n_expecs[(ix,iy)][1]))

norm = su.get_state().compute_norm()
print(norm)

n=0, tau=0.01, energy~0.000000:   1%|          | 3/278 [00:00<00:19, 14.40it/s]

2.4811640520882685e-18|-,+-,+-,+>
Ending Cycle1
-2.690933730486756e-18|-,0,0,+>
6.647677868015264e-19|-,+-,+-,+>
Ending Cycle2
-1.4076588010241515e-19|+,0,+-,+>
1.0962867872859863e-20|+,+-,0,+>
-1.6483979961684814e-20|+,+-,+-,+>
-3.754476439949827e-18|-,0,0,+>
3.947768494909605e-20|-,0,0,->
-2.0214955367129208e-18|-,0,+-,->
3.014867750075675e-18|-,+-,+-,+>
Ending Cycle3
-5.147489582975998e-19|+,0,+-,+>
-5.45068178941678e-20|+,+-,+-,+>
1.0831062366236802e-17|-,0,0,+>
7.57074880532869e-20|-,0,0,->
-2.852902508331458e-18|-,0,+-,->
-3.655938182268932e-18|-,+-,+-,+>
Ending Cycle4


n=0, tau=0.01, energy~0.000000:   3%|▎         | 7/278 [00:00<00:16, 16.56it/s]

1.1134122737443018e-20|+,0,0,->
-3.5261571686548953e-19|+,0,+-,+>
-1.721993744814485e-20|+,+-,0,+>
-5.0132712421173824e-20|+,+-,+-,+>
2.208013863995963e-18|-,0,0,+>
1.67914117889124e-19|-,0,0,->
-3.399289450447227e-18|-,0,+-,->
1.0368402446081385e-20|-,+-,0,->
2.4721611246433953e-18|-,+-,+-,+>
Ending Cycle5
-8.366488048212868e-19|+,0,+-,+>
-1.4898601211991563e-20|+,+-,0,+>
-4.591316550403315e-20|+,+-,+-,+>
9.227233320298202e-18|-,0,0,+>
4.962510871950253e-19|-,0,0,->
-6.1521063335740584e-18|-,0,+-,->
1.4484136888054943e-17|-,+-,+-,+>
Ending Cycle6
2.908695151580684e-20|+,0,0,->
-2.2924658772524627e-18|+,0,+-,+>
-3.8589308032096353e-20|+,+-,0,+>
-3.976639089345718e-20|+,+-,+-,+>
-4.986719666218777e-17|-,0,0,+>
6.765750669177418e-19|-,0,0,->
-6.828495888116361e-18|-,0,+-,->
-2.3982325311725744e-20|-,+-,0,->
4.9967845243611465e-17|-,+-,+-,+>
Ending Cycle7
2.175197919504413e-20|+,0,0,+>
7.574690141173284e-20|+,0,0,->
-3.6077155181618874e-18|+,0,+-,+>
-1.0006423748782682e-19|+,+-,0,+>
-4.20

n=0, tau=0.01, energy~0.000000:   4%|▍         | 11/278 [00:00<00:16, 16.31it/s]

...

-2.559694806962366e-13|+,0,0,+>
-2.1479559499368054e-14|+,0,0,->
4.948001239749996e-15|+,0,+-,+>
4.804104837543364e-15|+,+-,0,+>
-1.772296527976595e-15|+,+-,+-,+>
2.8497016417996255e-15|+,+-,+-,->
-4.882687514821802e-14|-,0,0,+>
-5.866745816335311e-15|-,0,0,->
5.942972194242288e-15|-,0,+-,->
6.27193593981377e-15|-,+-,0,->
-1.0800668114369425e-15|-,+-,+-,+>
1.2620475242459532e-13|-,+-,+-,->
Ending Cycle272
-3.017618402580019e-13|+,0,0,+>
-2.1851471782697118e-14|+,0,0,->
4.785144891867075e-15|+,0,+-,+>
4.791942878829826e-15|+,+-,0,+>
-2.525598398979487e-15|+,+-,+-,+>
2.854800957734322e-15|+,+-,+-,->
-4.8865384160565357e-14|-,0,0,+>
-5.861918045096043e-15|-,0,0,->
6.031624725851295e-15|-,0,+-,->
6.229759280822457e-15|-,+-,0,->
-1.0586267755655922e-15|-,+-,+-,+>
1.4931419686691696e-13|-,+-,+-,->
Ending Cycle273
-3.5605746849475526e-13|+,0,0,+>
-2.219732234553515e-14|+,0,0,->
4.803636388727251e-15|+,0,+-,+>
4.6950078519311766e-15|+,+-,0,+>
-2.8936924478998853e-15|+,+-,+-,+>
2.87025156393338

n=200, tau=0.01, energy~-1.859368: 100%|█████████▉| 277/278 [00:17<00:00, 16.34it/s]

-4.970630886439492e-13|+,0,0,+>
-2.2899176516771412e-14|+,0,0,->
4.764000256724111e-15|+,0,+-,+>
4.557478413685605e-15|+,+-,0,+>
-3.1509527855583506e-15|+,+-,+-,+>
2.9387665536932204e-15|+,+-,+-,->
-4.8919356438333355e-14|-,0,0,+>
-6.367766432596705e-15|-,0,0,->
5.964380392703633e-15|-,0,+-,->
6.069855148354943e-15|-,+-,0,->
-1.1092144232605145e-15|-,+-,+-,+>
2.4849798904831247e-13|-,+-,+-,->
Ending Cycle276
-5.880785748200001e-13|+,0,0,+>
-2.3266726470968755e-14|+,0,0,->
4.706854614670875e-15|+,0,+-,+>
4.6190369119391116e-15|+,+-,0,+>
-3.106230227801349e-15|+,+-,+-,+>
2.9435512270576704e-15|+,+-,+-,->
-4.8893829703476184e-14|-,0,0,+>
-6.485976264868538e-15|-,0,0,->
6.368177278100742e-15|-,0,+-,->
6.481473619627083e-15|-,+-,0,->
-1.0687068000880715e-15|-,+-,+-,+>
2.949747148006921e-13|-,+-,+-,->
Ending Cycle277
-6.963014796187964e-13|+,0,0,+>
-2.3625832428729458e-14|+,0,0,->
4.728179338640727e-15|+,0,+-,+>
4.416285621726505e-15|+,+-,0,+>
-2.996260173086741e-15|+,+-,+-,+>
2.730737692507

n=278, tau=0.01, energy~-1.900219: 100%|██████████| 278/278 [00:17<00:00, 15.74it/s]


(2, 2)
(0, 0): SZ=0.21, N=1.00
(1, 0): SZ=-0.14, N=1.00
(2, 0): SZ=0.14, N=1.00
(3, 0): SZ=-0.21, N=1.00
0.9999928215228403


In [26]:
psi1 = su.get_state()

for itsr in psi1:
    print(itsr.data.data)

su1 = SimpleUpdate(
    psi1,
    Ham,
    chi=128,  # boundary contraction bond dim for computing energy
    D = 4,
    compute_energy_every=100,
    compute_energy_per_site=False,
    keep_best=True,
    ordering = 'sort',
    gauge_smudge = 1e-6,
    callback=callback#,
    #gate_opts = {'cutoff': 1e-6}
)
tau = 0.01
su1.evolve(1, tau=tau)

second_dense = min(Lx, Ly)>1
sz_expecs = su1.get_state().compute_local_expectation(sz_ops, return_all=True, second_dense=second_dense,normalized=True)
n_expecs = su1.get_state().compute_local_expectation(n_ops, return_all=True, second_dense=second_dense,normalized=True)

print(su1.get_state()[0,0].shape)
for ix, iy in itertools.product(range(Lx), range(Ly)):
    print("(%i, %i): SZ=%.2f, N=%.2f"%(ix,iy,sz_expecs[(ix,iy)][0]/sz_expecs[(ix,iy)][1], n_expecs[(ix,iy)][0]/n_expecs[(ix,iy)][1]))

norm = su1.get_state().compute_norm()
print(norm)


n=0, tau=0.01, energy~-0.115293: 100%|██████████| 1/1 [00:00<00:00,  7.27it/s]

[-0.09899956  0.          0.         -0.09949088 -1.00486561  0.
  0.          0.00999741]
[ 9.90067826e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.66397683e-21  9.90018325e-02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -2.16058167e-17  9.82587235e-04
  0.00000000e+00  0.00000000e+00 -9.87587018e-04  2.65248113e-17
  0.00000000e+00  9.80046210e-02  9.85057004e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -6.97948764e-04  6.94547727e-04  0.00000000e+00
 -9.85873202e-04 -1.31157586e-19  1.31863282e-19 -9.39316310e-20
  9.84958503e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.04271564e-18  0.00000000e+00 -9.89944913e-03  0.00000000e+00
  0.00000000e+00  2.08552111e-24  0.00000000e+00  9.87491063e-03
  0.00000000e+00  9.87467063e-03  0.00000000e+00 -2.34018531e-20
  0.00000000e+00 -9.95013534e-01 -9.90032584e-03  0.00000000e+00
 -9.91675546e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00

n=1, tau=0.01, energy~-0.221545: 100%|██████████| 1/1 [00:00<00:00,  3.90it/s]


(2, 2)
(0, 0): SZ=0.50, N=1.00
(1, 0): SZ=-0.50, N=1.00
(2, 0): SZ=0.50, N=1.00
(3, 0): SZ=-0.50, N=1.00
0.9999958480439124


In [ ]:
psi1 = su.get_state()
state = psi1.contract(all)
#state.data.data[abs(state.data.data)<1e-10] = 0
tsr = state.data.to_sparse()
for iblk in tsr:
    print_block(iblk)

norm = psi1.compute_norm()

for where, G in n_ops.items():
    ket = psi1.copy()
    ket.add_tag("KET")
    bra = ket.H
    bra.retag({"KET": "BRA"})
    bra.mangle_inner_("*")
    ket.gate_(G, where, contract=True)
    tn = ket & bra
    out = tn.contract(all)
    print(where, out/norm)